First Attempt at Problem... Will use Random Forest Regressor...
Let's see where it goes.

In [220]:
import pandas as pd
import numpy as np

In [221]:
train_data = pd.read_csv('Dataset/Train.csv')
test_data = pd.read_csv('Dataset/Test.csv')

In [222]:
train_data.head(5)

,Employee_ID,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,...,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate
0,EID_23371,F,42.0,4,Married,Franklin,IT,Conceptual,4.0,4,...,type2,3.0,4,0.7516,1.8688,2.0,4,5,3,0.1841
1,EID_18000,M,24.0,3,Single,Springfield,Logistics,Analytical,5.0,4,...,type2,4.0,3,-0.9612,-0.4537,2.0,3,5,3,0.0670
2,EID_3891,F,58.0,3,Married,Clinton,Quality,Conceptual,27.0,3,...,type2,1.0,4,-0.9612,-0.4537,3.0,3,8,3,0.0851
3,EID_17492,F,26.0,3,Single,Lebanon,Human Resource Management,Behavioral,4.0,3,...,type2,1.0,3,-1.8176,-0.4537,NaN,3,7,3,0.0668
4,EID_22534,F,31.0,1,Married,Springfield,Logistics,Conceptual,5.0,4,...,type3,3.0,1,0.7516,-0.4537,2.0,2,8,2,0.1827


In [223]:
#checking which columns have NaN values
c = 0
for i in train_data.columns:
    if (train_data[i].isnull().values.any()) == True:
        print(i, c)
    c += 1

Age 2
Time_of_service 8
Pay_Scale 13
Work_Life_balance 15
VAR2 17
VAR4 19


In [243]:
X = train_data.iloc[:, 1:-1].values
y = train_data.iloc[:,-1].values
test_X = test_data.iloc[:, 1:].values
print(y)

[0.1841 0.067  0.0851 ... 0.7847 0.4162 0.091 ]


In [244]:
from sklearn.impute import SimpleImputer
impute = SimpleImputer(missing_values = np.nan, strategy = 'mean')
impute = impute.fit(X[:, [1, 7, 12, 14, 16, 18]])
X[:, [1, 7, 12, 14, 16, 18]] = impute.transform(X[:, [1, 7, 12, 14, 16, 18]])
# imputing on test data
test_impute = SimpleImputer(missing_values = np.nan, strategy = 'mean')
test_impute = test_impute.fit(X[:, [1, 7, 12, 14, 16, 18]])
test_X[:, [1, 7, 12, 14, 16, 18]] = test_impute.transform(test_X[:, [1, 7, 12, 14, 16, 18]])

In [247]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
category_list = [0, 3, 4, 5, 6, 11, 12, 13]
for i in category_list:
    X[:, i] = labelencoder_X.fit_transform(X[:, i])
    test_X[:, i] = labelencoder_X.fit_transform(test_X[:, i])
    onehotencoder = OneHotEncoder(categorical_features=[i])
    X = onehotencoder.fit_transform(X).toarray()
    test_X = onehotencoder.fit_transform(test_X).toarray()
#labelencoder_y = LabelEncoder()
#y = labelencoder_y.fit_transform(y)
import warnings
warnings.filterwarnings('ignore')

In [276]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 1000, random_state=0)
regressor.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [277]:
rows = zip(test_data['Employee_ID'].values, regressor.predict(test_X))
import csv
with open('my_submission_3.csv', "w") as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)

In [278]:
df = pd.read_csv('my_submission_2.csv', header=None)
df.rename(columns={0: 'Employee_ID', 1: 'Attrition_rate'}, inplace=True)
df.to_csv('my_submission_3.csv', index=False) # save to new csv file